In [135]:
!pip install elasticsearch[async]==7.17.0 -q
!pip install elasticsearch-dsl==7.4.0 -q

In [264]:
import sys

sys.path.append('/app')

In [265]:
from datetime import datetime
from elasticsearch import Elasticsearch
from elasticsearch_dsl import Search
import pandas as pd
import ast
import json

In [266]:
from memlish.config import ESTag

In [267]:
client = Elasticsearch("http://ec2-34-245-85-126.eu-west-1.compute.amazonaws.com:9200")
index = "fluentd-*"

In [268]:
# client.delete_by_query(index=[index], body={"query": {"match_all": {}}}, size=1000000)

In [269]:
# client.delete_by_query(index=[index], body={"query": {
#     "bool": {
#       "must": {
#         "match": {
#           "log": ESTag.TIMELOG
#         }
#       }
#     }
#   }}, size=1000000)

In [270]:
# time_logs = []
# time_log_search = Search(using=client, index=index).query("multi_match", query=ESTag.TIMELOG, fields=['log'])

# for hit in time_log_search.scan():
#     _hit = {}
#     _hit = ast.literal_eval(hit.to_dict()['log'])
#     time_logs.append(_hit)

In [271]:
client.info()

{'name': 'elastic',
 'cluster_name': 'docker-cluster',
 'cluster_uuid': 'oyf18TppTqyj5cwfBqvswg',
 'version': {'number': '7.0.1',
  'build_flavor': 'default',
  'build_type': 'docker',
  'build_hash': 'e4efcb5',
  'build_date': '2019-04-29T12:56:03.145736Z',
  'build_snapshot': False,
  'lucene_version': '8.0.0',
  'minimum_wire_compatibility_version': '6.7.0',
  'minimum_index_compatibility_version': '6.0.0-beta1'},
 'tagline': 'You Know, for Search'}

In [272]:
answ_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_ANSWER, fields=['log'])
choice_search = Search(using=client, index=index).query("multi_match", query=ESTag.INLINE_CHOICE, fields=['log'])

In [273]:
#cand_id - is guid. Example: 3940172d-fda5-4e02-b284-6f5e2074454c 
def find_choice_for_answer(cand_id, choices):
    for i in choices:
        choice_id = i['update']['result_id'].split('_')[1]
        if choice_id == cand_id:
            return i

In [274]:
choice_logs = []
rows = []

for hit in choice_search.scan():
    _hit = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit['update'] = json.loads(_hit['update'])
    _hit["timestamp"] = hit["@timestamp"]
    choice_logs.append(_hit)

for hit in answ_search.scan():
    result_row = {}
    _hit = ast.literal_eval(hit.to_dict()['log'])
    _hit["answer_timestamp"] = hit["@timestamp"]
    if len(_hit['template_options']) > 0:
        _hit_template_uniq_id = _hit['template_options'][0]['id'].split('_')[1]
        possible_choice = find_choice_for_answer(_hit_template_uniq_id, choice_logs)
    
    update = json.loads(_hit['update'])
    
    if possible_choice is not None:
        choice_update = possible_choice['update']
        result_row["template_choice"] = choice_update["result_id"]
        result_row["choice_timestamp"] = possible_choice['timestamp']
    
    result_row["answer_timestamp"] = _hit["answer_timestamp"]
    result_row["id"] = update["id"]
    result_row["user_id"] = update["from"]["id"]
    result_row["first_name"] = update["from"]["first_name"]
    result_row["last_name"] = update["from"].get("last_name", "")
    
    if update["from"].get("username", None) is not None:
        result_row["username"] = update["from"]["username"]
    else:
        result_row["username"] = update["from"]["id"]
        
    result_row["language_code"] = update["from"]["language_code"]
    result_row["chat_type"] = update["chat_type"]
    result_row["query"] = update["query"]
    result_row["candidates"] = _hit["template_options"]
    
    rows.append(result_row)

In [275]:
logs_df = pd.DataFrame(rows)

In [276]:
banned_users = ['daniel_kovalenko', 'ParadiseN', 'vprypeshniuk', 'kilianovski', 'ostap_hembara', 'CheshireDog1865', 'dasha_hatsosh']

banned_mask = logs_df['username'].apply(lambda x: x not in banned_users)

with_choice_mask = logs_df['template_choice'].notna()

selection_mask = banned_mask & with_choice_mask

In [277]:
logs_df[selection_mask]['username'].value_counts()

sergej_havrylyuk      4
Three_part_baton      3
Sami_ykb              1
1270542385            1
Player_unknown_777    1
imameme_guy           1
MarynaMarchenko       1
Anvilondre            1
Yeabikibra            1
Name: username, dtype: int64

In [278]:
logs_df[selection_mask]['username'].value_counts().shape

(9,)

In [232]:
logs_df[selection_mask]

,answer_timestamp,id,user_id,first_name,last_name,username,language_code,chat_type,query,candidates,template_choice,choice_timestamp
74,2022-02-16T08:20:16.000000000+00:00,3315249643607453854,5066859004,ֆǟʍɨ 👑,,Sami_ykb,en,sender,meme,[{'id': '0_3483319e-deb3-4f84-a0ce-d83ce144df0...,0_3483319e-deb3-4f84-a0ce-d83ce144df06,2022-02-16T08:20:30.000000000+00:00
211,2022-02-17T14:50:14.000000000+00:00,5456937993646486604,1270542385,A,T,1270542385,en,group,жить до 160!,[{'id': '0_53707c81-bb81-4cf3-b06e-629baa2354a...,32_53707c81-bb81-4cf3-b06e-629baa2354a5,2022-02-17T14:51:27.000000000+00:00
335,2022-02-17T17:01:21.000000000+00:00,3882595378931702022,903987181,✯͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͜𐌟✯͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚͚...,,Player_unknown_777,ru,sender,а у меня длинее,[{'id': '0_5d56c524-428d-47cb-92cf-71cd8739e34...,8_5d56c524-428d-47cb-92cf-71cd8739e341,2022-02-17T17:01:39.000000000+00:00
385,2022-02-16T12:57:27.000000000+00:00,5189230960696926468,1208211984,meme,,imameme_guy,en,private,fever,[{'id': '0_1b954613-2370-4c64-925b-745082c9a96...,43_1b954613-2370-4c64-925b-745082c9a962,2022-02-16T12:57:46.000000000+00:00
392,2022-02-16T13:17:30.000000000+00:00,2189866687606060220,509868070,Марина,Марченко,MarynaMarchenko,uk,group,шабака,[{'id': '0_8471e7f3-0373-40ce-a3af-51b401c965c...,29_8471e7f3-0373-40ce-a3af-51b401c965cc,2022-02-16T13:17:48.000000000+00:00
487,2022-02-16T18:52:10.000000000+00:00,2127971286848656459,495456924,Sergej,Havrylyuk,sergej_havrylyuk,ru,private,це прекрасно,[{'id': '0_976dc3d3-7b8d-4b11-86b3-69dea6095b1...,3_976dc3d3-7b8d-4b11-86b3-69dea6095b1a,2022-02-16T18:52:22.000000000+00:00
500,2022-02-16T18:55:15.000000000+00:00,2127971288307855106,495456924,Sergej,Havrylyuk,sergej_havrylyuk,ru,private,до 15 або після 20 фантастично,[{'id': '0_2a93e3bb-34f4-4427-b132-cd3461ce05a...,8_2a93e3bb-34f4-4427-b132-cd3461ce05a5,2022-02-16T18:55:28.000000000+00:00
511,2022-02-16T19:02:52.000000000+00:00,2127971287875037999,495456924,Sergej,Havrylyuk,sergej_havrylyuk,ru,private,скучив,[{'id': '0_12da6813-2223-4f27-8b5d-e58f71f4a9e...,44_12da6813-2223-4f27-8b5d-e58f71f4a9eb,2022-02-16T19:03:20.000000000+00:00
532,2022-02-16T20:47:05.000000000+00:00,2127971288214266281,495456924,Sergej,Havrylyuk,sergej_havrylyuk,ru,private,Назар суперовий,[{'id': '0_66dbf3f0-842d-48f0-b91c-2dd792853de...,39_66dbf3f0-842d-48f0-b91c-2dd792853de4,2022-02-16T20:47:21.000000000+00:00
648,2022-02-16T09:35:24.000000000+00:00,1660455432584386481,386604907,Роман,🐼,Anvilondre,en,private,симсыыы,[{'id': '0_d2aef5c5-1ee5-41bb-8b1f-1ba46d5bb85...,47_d2aef5c5-1ee5-41bb-8b1f-1ba46d5bb85d,2022-02-16T09:35:46.000000000+00:00


In [263]:
logs_df[selection_mask]['query']

74                               meme
211                      жить до 160!
335                   а у меня длинее
385                             fever
392                            шабака
487                      це прекрасно
500    до 15 або після 20 фантастично
511                            скучив
532                   Назар суперовий
648                           симсыыы
653                         I'm sorry
657                            not me
658                              sexy
Name: query, dtype: object